<a href="https://colab.research.google.com/github/lokolope24/Imersao_IA_Alura/blob/main/HelpDeskIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bibliotecas

Instalando o Gradio e Importando as demais bibliotecas para usar no projeto


In [1]:
pip install -U -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 

In [2]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
import numpy as np
import gradio as gr

# API Key e Modelo Gemini

Utilizando a API Key do Google IA Studio e o Modelo de LLM Gemini (gemini-1.0-pro), abaixo está a configuração do modelo generativo e do embedding:


In [3]:
api_key = userdata.get("secret_key")
GOOGLE_API_KEY=api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
generation_config = {
    "candidate_count" : 1,
    "temperature": 0.5,
}

safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL":"BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE",
}

model_generative_AI = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config, safety_settings=safety_settings)

In [5]:
#Modelo de Embedding utilizado:
model_embedding = "models/embedding-001"

# Importando Dados

Importando o arquivo excel que iremos usar como a base de dados do projeto HelpDesk e realizando o pré processamento dos dados.

  O arquivo contém um histórico de atendimento dentro da área da TI voltado a infraestrutura, conténdo como pontos focais da análise as colunas "Assunto", "Diagnóstico", "Causa" e "Ação".
  Realizamos também um pré-processamento dos dados e limpando as possiveis sujeiras que estão no arquivo.
  

In [6]:
# Lê o arquivo Excel usando pandas
df_helpdesk = pd.read_excel('helpdesk.xlsx')

#Renomeia as colunas na ordem que foram importadas:
df_helpdesk.columns = ["assunto", "diagnostico", "causa", "acao"]

# Remove registros onde qualquer uma das colunas especificadas tem um valor nulo
df_helpdesk = df_helpdesk.dropna(subset=["assunto", "diagnostico", "causa", "acao"])

#Unificando as colunas em apenas uma.
df_helpdesk["diagnostico_causa_acao"] = "Diagnostico : "+ df_helpdesk["diagnostico"] + " Causa : " + df_helpdesk["causa"] + " Acao : " + df_helpdesk["acao"]

#deletando as demais colunas para organização
df_helpdesk = df_helpdesk.drop(columns=["diagnostico", "causa", "acao"])

df_helpdesk = df_helpdesk.head(100)



# Embedding


1.    Criando a função de embedding do conteúdo do DataFrame
2.    Criando o embedding da consulta para comparar com o conteúdo do DataFrame



In [7]:
def embedding_function(title, text):

  return genai.embed_content(model=model_embedding, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [8]:
df_helpdesk["embeddings"] = df_helpdesk.apply(lambda row:
                                              embedding_function(row["assunto"],
                                                                 row["diagnostico_causa_acao"]), axis=1)

In [9]:
def gerar_consulta(consulta, base, model):

  #Gera o embedding da consulta:
  embedding_da_consulta = genai.embed_content(model=model_embedding, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df_helpdesk["embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)

  return df_helpdesk.iloc[indice]["diagnostico_causa_acao"]

#Função Interativa

Criando uma função para utilizar o modelo de Gemini + embeddings


In [10]:
def consulta_interativa(consulta):
    trecho = gerar_consulta(consulta, df_helpdesk, model_embedding)
    prompt = f"Responda como um especialista em Infraestrutura que está auxiliando o funcionário novo que está com a seguinte dúvida: {consulta} tendo como informações a seguinte base de referência {trecho}"
    response = model_generative_AI.generate_content(prompt)
    return response.text


#Interface gráfica (Gradio)

utilizando o Gradio para criar o chatbot e ter o prompt de perguntas e respostas.

In [11]:
interface = gr.Interface(fn=consulta_interativa,
                         inputs="text",
                         outputs="text",
                         title="HelpDesk IAssist",
                         description="Diga, em que posso lhe auxiliar?")
interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f1dc584331215ed5ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f1dc584331215ed5ce.gradio.live
